In [1]:
import os
from collections import defaultdict
from pathlib import Path
from typing import Any, List, Callable, Union

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm


def one_hot_encode(
        sequence: str,
        alphabet: str = "ACGT",
        neutral_alphabet: str = "N",
        neutral_value: Any = 0,
        dtype=np.float32
    ) -> np.ndarray:
    """One-hot encode sequence."""

    def to_uint8(s):
        return np.frombuffer(s.encode("ascii"), dtype=np.uint8)

    lookup = np.zeros([np.iinfo(np.uint8).max, len(alphabet)], dtype=dtype)
    lookup[to_uint8(alphabet)] = np.eye(len(alphabet), dtype=dtype)
    lookup[to_uint8(neutral_alphabet)] = neutral_value
    lookup = lookup.astype(dtype)
    return lookup[to_uint8(sequence)]


class WindowedGenomeDataset:

    def __init__(
            self,
            target_path: str,
            fasta_path: str,
            window_size: int = None,
            chromosomes: List[str] = None,
            dtype: np.float32 = None
        ):
        self.target_path = target_path
        self.fasta_path = fasta_path
        self.chromosomes = chromosomes
        self.dtype = dtype

        df = pd.read_table(target_path, header=None, sep="\t")
        c = df[0].isin(chromosomes) if chromosomes else np.array(len(df)*[True])
        self.windows = df.loc[c, :2].values
        self.targets = df.loc[c, 3:].values.astype(self.dtype)

        _, start, stop = self.windows[0]
        self.window_size = window_size or (stop - start)

        self.input_shape = (self.window_size, 4)
        self.output_shape = (self.targets.shape[1],)

        self.genome = None

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, item: Union[int, slice]):
        if not self.genome:
            from pyfaidx import Fasta
            self.genome = Fasta(self.fasta_path)

        chrom = self.windows[item, 0]
        start = self.windows[item, 1]
        stop = self.windows[item, 2]

        labels = self.targets[item]

        if isinstance(item, slice):
            seq = ""
            for chr, i, j in zip(chrom, start, stop):
                seq += self.genome[chr][i:j].seq.upper()

            inputs = one_hot_encode(seq)
            inputs = inputs.reshape(len(chrom), self.window_size, 4)

            item = range(item.start or 0, item.stop or len(self), item.step or 1)
            item = np.array(list(item), dtype=np.int32)
        else:
            seq = self.genome[chrom][start:stop].seq.upper()
            inputs = one_hot_encode(seq)

            item = item if item >= 0 else len(self) + item

        output = {
            "inputs": inputs,
            "labels": labels,
            "meta": {
                "id": item,
                "chrom": chrom,
                "start": start,
                "stop": stop
            }
        }
        return output

    
    def to_tfrecord(self, path: str, indices: List[int] = None):

        def serialize_example(id, inputs, labels, chrom, start, stop):

            inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
            inputs = tf.io.serialize_tensor(inputs).numpy()
            labels = tf.convert_to_tensor(labels, dtype=tf.float32)
            labels = tf.io.serialize_tensor(labels).numpy()

            chrom = chrom.encode("utf-8")

            feature = {
                "id": tf.train.Feature(int64_list=tf.train.Int64List(value=[id])),
                "inputs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs])),
                "labels": tf.train.Feature(bytes_list=tf.train.BytesList(value=[labels])),
                "chrom": tf.train.Feature(bytes_list=tf.train.BytesList(value=[chrom])),
                "start": tf.train.Feature(int64_list=tf.train.Int64List(value=[start])),
                "stop": tf.train.Feature(int64_list=tf.train.Int64List(value=[stop]))
            }

            example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
            return example_proto.SerializeToString()

        def generator():
            for i in tqdm(indices or range(len(self))):
                output = self[i]
                yield serialize_example(
                    id=output["meta"]["id"],
                    inputs=output["inputs"],
                    labels=output["labels"],
                    chrom=output["meta"]["chrom"],
                    start=output["meta"]["start"],
                    stop=output["meta"]["stop"]
                )

        serialized_features_dataset = tf.data.Dataset.from_generator(
            generator,
            output_types=tf.string,
            output_shapes=()
        )

        writer = tf.data.experimental.TFRecordWriter(path)
        writer.write(serialized_features_dataset)


In [2]:
DATA_DIR = "/home/chandana/projects/biccn/data/"
SAVE_DIR = "/home/chandana/projects/biccn/models/"

In [3]:
items = [
    ("train", [2, 4, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, "X"]),
    ("valid", [7, 9]),
    ("test", [1, 3, 5])
]

save = False

datasets = {}

for name, chroms in items:
    datasets[name] = WindowedGenomeDataset(
        target_path=f"{DATA_DIR}/targets.bed",
        fasta_path=f"{DATA_DIR}/GRCm38.fa",
        chromosomes=[f"chr{i}" for i in chroms],
        window_size=999,
        dtype=np.float32
    )
    if save:
        datasets[name].to_tfrecord(f"/home/chandana/projects/biccn/data/{name}.tfrec")

# Data

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
AUTO = tf.data.experimental.AUTOTUNE
THRESHOLD = 0.0006867924257022952
BATCH_SIZE = 128

        

def parse_tfrecord(example):
    feature_description = {
        "id": tf.io.FixedLenFeature([], tf.int64),
        "inputs": tf.io.FixedLenFeature([], tf.string),
        "labels": tf.io.FixedLenFeature([], tf.string),
        "chrom": tf.io.FixedLenFeature([], tf.string),
        "start": tf.io.FixedLenFeature([], tf.int64),
        "stop": tf.io.FixedLenFeature([], tf.int64)
    }
    
    record = tf.io.parse_single_example(example, feature_description)
    record["inputs"] = tf.io.parse_tensor(record["inputs"], out_type=tf.float32)
    record["labels"] = tf.io.parse_tensor(record["labels"], out_type=tf.float32)

    inputs = tf.reshape(record["inputs"] , [999, 4])
    labels = tf.reshape(record["labels"], [33,])

    return inputs, labels

def load_dataset(filenames):

    records = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    records = records.map(parse_tfrecord, num_parallel_calls=AUTO)

    def data_augment(inputs, labels):
        return inputs, tf.where(labels > THRESHOLD, 1.0, 0.0)


    return records.map(data_augment, num_parallel_calls=AUTO)


def get_dataset(batch_size: int, name: str):

    if name == "train":
        dataset = load_dataset([DATA_DIR + "/train.tfrec"])
        dataset = dataset.shuffle(10000)
        dataset = dataset.repeat()
    elif name == "valid":
        dataset = load_dataset([DATA_DIR + "/valid.tfrec"])
        dataset = dataset.repeat()
    elif name == "test":
        dataset = load_dataset([DATA_DIR + "/test.tfrec"])

    dataset = dataset.batch(batch_size).prefetch(AUTO) 

    return dataset

In [6]:
train_size = len(datasets['train'])
valid_size = len(datasets['valid'])
test_size = len(datasets['test'])

total = train_size + valid_size + test_size

print("Train: ", train_size / total)
print("Validation: ", valid_size / total)
print("Validation: ", test_size / total)

Train:  0.7026630169507466
Validation:  0.10160312906924136
Validation:  0.19573385398001197


In [7]:
train_ds = get_dataset(BATCH_SIZE, name="train")
valid_ds = get_dataset(BATCH_SIZE, name="valid")
test_ds = get_dataset(BATCH_SIZE, name="test")

2022-05-10 16:03:49.023627: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-10 16:03:49.796372: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9653 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5


# Model

In [23]:
class RevCompConv1D(tf.keras.layers.Conv1D):
  """
  Implement forward and reverse-complement filter convolutions
  for 1D signals. It takes as input either a single input or two inputs
  (where the second input is the reverse complement scan). If a single input,
  this performs both forward and reverse complement scans and either merges it
  (if concat=True) or returns a separate scan for forward and reverse comp.
  """
  def __init__(self, *args, concat=False, **kwargs):
    super(RevCompConv1D, self).__init__(*args, **kwargs)
    self.concat = concat


  def call(self, inputs, inputs2=None):

    if inputs2 is not None:
      # create rc_kernels
      rc_kernel = self.kernel[::-1,::-1,:]

      # convolution 1D
      outputs = self._convolution_op(inputs, self.kernel)
      rc_outputs = self._convolution_op(inputs2, rc_kernel)

    else:
      # create rc_kernels
      rc_kernel = tf.concat([self.kernel, self.kernel[::-1,::-1,:]], axis=-1)

      # convolution 1D
      outputs = self._convolution_op(inputs, rc_kernel)

      # unstack to forward and reverse strands
      outputs = tf.unstack(outputs, axis=2)
      rc_outputs = tf.stack(outputs[self.filters:], axis=2)
      outputs = tf.stack(outputs[:self.filters], axis=2)

    # add bias
    if self.use_bias:
      outputs = tf.nn.bias_add(outputs, self.bias)
      rc_outputs = tf.nn.bias_add(rc_outputs, self.bias)

    # add activations
    if self.activation is not None:
      outputs = self.activation(outputs)
      rc_outputs = self.activation(rc_outputs)

    if self.concat:
      return tf.concat([outputs, rc_outputs], axis=-1)
    else:
      return outputs, rc_outputs


def dilated_residual_block(input_layer, filter_size, rates):
    num_filters = input_layer.shape.as_list()[-1]
    nn = tf.keras.layers.Conv1D(
        filters=num_filters,
        kernel_size=filter_size,
        activation=None,
        use_bias=False,
        padding='same',
        dilation_rate=rates[0],
    )(input_layer)
    nn = tf.keras.layers.BatchNormalization()(nn)

    for f in rates[1:]:
        nn = tf.keras.layers.Activation('relu')(nn)
        nn = tf.keras.layers.Dropout(0.1)(nn)
        nn = tf.keras.layers.Conv1D(
            filters=num_filters,
            kernel_size=filter_size,
            strides=1,
            activation=None,
            use_bias=False,
            padding='same',
            dilation_rate=f
        )(nn)

    nn = tf.keras.layers.BatchNormalization()(nn)
    nn = tf.keras.layers.add([input_layer, nn])
    return tf.keras.layers.Activation('relu')(nn)


def residualbind(input_shape, output_shape, activation='exponential', num_units=[128, 256, 512, 512], rc=False):

    # input layer
    inputs = tf.keras.layers.Input(shape=input_shape)

    # layer 1
    if rc:
        nn = RevCompConv1D(filters=num_units[0], kernel_size=19, use_bias=False, padding='same', concat=True)(inputs)
    else:
        nn = tf.keras.layers.Conv1D(
            filters=num_units[0],
            kernel_size=19,
            strides=1,
            activation=None,
            use_bias=False,
            padding='same'
        )(inputs)

    nn = tf.keras.layers.BatchNormalization()(nn)
    nn = tf.keras.layers.Activation(activation)(nn)
    nn = tf.keras.layers.Dropout(0.1)(nn)

    # dilated residual block
    nn = dilated_residual_block(nn, filter_size=3, rates=[1, 2, 4, 8])
    nn = tf.keras.layers.MaxPooling1D(pool_size=10)(nn) # 500
    nn = tf.keras.layers.Dropout(0.2)(nn)

    # layer 2
    nn = tf.keras.layers.Conv1D(
        filters=num_units[1],
        kernel_size=7,
        strides=1,
        activation=None,
        use_bias=False,
        padding='same',
    )(nn)
    nn = tf.keras.layers.BatchNormalization()(nn)
    nn = tf.keras.layers.Activation('relu')(nn)
    nn = tf.keras.layers.Dropout(0.1)(nn)

    # dilated residual block
    nn = dilated_residual_block(nn, filter_size=3, rates=[1, 2, 4])
    nn = tf.keras.layers.MaxPooling1D(pool_size=10)(nn) # 50
    nn = tf.keras.layers.Dropout(0.2)(nn)

  # layer 2
    nn = tf.keras.layers.Conv1D(
        filters=num_units[2],
        kernel_size=7,
        strides=1,
        activation=None,
        use_bias=False,
        padding='same',
    )(nn)
    nn = tf.keras.layers.BatchNormalization()(nn)
    nn = tf.keras.layers.Activation('relu')(nn)

    nn = tf.keras.layers.GlobalAveragePooling1D()(nn) # 1
    nn = tf.keras.layers.Dropout(0.3)(nn)

    # Fully-connected NN
    nn = tf.keras.layers.Flatten()(nn)
    nn = tf.keras.layers.Dense(num_units[3], activation=None, use_bias=False)(nn)
    nn = tf.keras.layers.BatchNormalization()(nn)
    nn = tf.keras.layers.Activation('relu')(nn)
    nn = tf.keras.layers.Dropout(0.5)(nn)

    # output layer
    logits = tf.keras.layers.Dense(output_shape, activation='linear', use_bias=True)(nn)
    outputs = tf.keras.layers.Activation('sigmoid')(logits)

    # create and return model
    return tf.keras.Model(inputs=inputs, outputs=outputs)

# Training

In [27]:
model = residualbind(
    input_shape=(999, 4),
    output_shape=33,
    activation="exponential", 
    num_units=[128, 256, 512, 512],
    rc=False
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.0),
    metrics=[
        tf.keras.metrics.AUC(curve="ROC", name="auroc"),
        tf.keras.metrics.AUC(curve="PR", name="aupr")
    ]
)

In [28]:
train_steps = train_size // BATCH_SIZE
valid_steps = valid_size // BATCH_SIZE

history = model.fit(
        train_ds,
        epochs=2,
        steps_per_epoch=train_steps,
        validation_data=valid_ds,
        validation_steps=valid_steps,
        callbacks=[
             tf.keras.callbacks.EarlyStopping(
                monitor="val_aupr",
                patience=10,
                verbose=1,
                mode="max",
                restore_best_weights=True
            ),
             tf.keras.callbacks.ReduceLROnPlateau(
                monitor="val_aupr",
                factor=0.2,
                patience=3,
                min_lr=1e-7,
                mode="max",
                verbose=1
            )
        ]
    )

Epoch 1/2
100/100 [==============================] - 14s 99ms/step - loss: 0.6911 - auroc: 0.6761 - aupr: 0.5988 - val_loss: 0.4691 - val_auroc: 0.5641 - val_aupr: 0.1743 - lr: 0.0010
Epoch 2/2
100/100 [==============================] - 9s 93ms/step - loss: 0.6029 - auroc: 0.7436 - aupr: 0.6607 - val_loss: 0.5598 - val_auroc: 0.5876 - val_aupr: 0.1969 - lr: 0.0010


In [25]:
model.load_weights("/home/chandana/projects/biccn/models/resbind_weights.h5")

In [26]:
model.evaluate(test_ds, steps=test_size // BATCH_SIZE)

2022-05-10 16:10:22.508248: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


3771/3771 [==============================] - 80s 20ms/step - loss: 0.5971 - auroc: 0.7819 - aupr: 0.7208


[0.5970627665519714, 0.7819393873214722, 0.7208306789398193]